In [1]:
from pathlib import Path
import os

yr = 2023
d = 14

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import numpy as np
def format_input(inp):
  formatted_input = []
  for l in inp.split('\n'):
    formatted_input.append(list(l))
  return np.array(formatted_input, dtype=str)

In [3]:
def roll_single(arr, rev=False):
  '''
  Roll the rocks in a single row/column
  '''
  roll_to_loc = 0

  if rev:
    arr = list(reversed(arr))
  for i in range(len(arr)):
    if arr[i]=='#':
      roll_to_loc=i+1
    elif arr[i]=='O':
      arr[i] = '.'
      arr[roll_to_loc] = 'O'
      roll_to_loc += 1
  if rev:
    return list(reversed(arr))
  else:
    return arr

In [4]:
def roll(arr, dir='up', cp=True):
  '''
  Roll all the rocks
  '''
  if cp:
    import copy
    arr = copy.deepcopy(arr)
  if dir == 'up':
    for i in range(arr.shape[1]):
      arr[:,i] = roll_single(arr[:,i])
  elif dir == 'down':
    for i in range(arr.shape[1]):
      arr[:,i] = roll_single(arr[:,i], rev=True)
  elif dir == 'left':
    for i in range(arr.T.shape[0]):
      arr.T[:,i] = roll_single(arr.T[:,i])
  elif dir == 'right':
    for i in range(arr.T.shape[0]):
      arr.T[:,i] = roll_single(arr.T[:,i], rev=True)
  return arr

In [5]:
def calculate_load(arr):
  '''
  Calculate the load of the current rock configuration
  '''
  s = 0
  for i in range(arr.shape[0]):
    s += (sum([x=='O' for x in arr[arr.shape[0]-1-i]])) * (i+1)
  return s

In [6]:
def roll_north_and_calculate_load(arr):
  return calculate_load(roll(arr))

In [7]:
def spin_cycle(arr, cp=True):
  '''
  Do a full spin cycle
  '''
  return roll(roll(roll(roll(arr, dir='up', cp=cp), dir='left', cp=False), dir='down', cp=False), dir='right', cp=False)

In [8]:
def arr_to_str(arr):
  '''
  Convert the array that represents the platform to a string
  '''
  return ''.join([''.join(x) for x in arr])

def arr_to_hash(arr):
  '''
  Hash the platform array with arr_to_str() as an intermediate step
  '''
  return hash(arr_to_str(arr))

In [9]:
def find_spin_cycle_loop(arr, cp=True):
  '''
  Run the spin cycle until you get back
  to a configuration you've seen before

  return the hashes of the array after each cycle,
  as well as the arrays themselves
  '''
  if cp:
    import copy
    arr = copy.deepcopy(arr)
  hashes = [arr_to_hash(arr)]
  hash_2_arr = {arr_to_hash(arr): arr}
  while(len(set(hashes))==len(hashes)):
    arr = spin_cycle(arr, cp=True)
    hashes.append(arr_to_hash(arr))
    hash_2_arr[arr_to_hash(arr)] = arr
  return hashes, hash_2_arr


In [10]:
def spin_cycle_and_calculate_load(arr, n, cp=True):
  '''
  Figure out how long it takes to get into a cycle
  and use some modular arithmetic to determine what the
  configuration will be like after n spin cycles

  return the load of that configuration
  '''
  if cp:
    import copy
    arr = copy.deepcopy(arr)
  hashes, hash_2_arr = find_spin_cycle_loop(arr, cp=False)
  n_before_loop = hashes.index(hashes[-1]) - 1
  loop_size = len(hashes) - n_before_loop - 2
  n_hash = hashes[(n_before_loop + (n-n_before_loop)%loop_size)]
  n_arr = hash_2_arr[n_hash]
  return calculate_load(hash_2_arr[n_hash])

In [11]:
import time

t = time.time()

formatted_input = format_input(inp)

print(roll_north_and_calculate_load(formatted_input))
print(spin_cycle_and_calculate_load(formatted_input, 1000000000))

print('\nRUNTIME: ', time.time()-t)

112048
105606

RUNTIME:  3.393922805786133
